### Logistic Regression

This model is built using simple Logistic Regression. Text data is transformed into feature vectors using TFIDF on both words as well as characters.

In [1]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [5]:
def load_data():
    '''
    Load data from csv files.
    :return train: Pandas dataframe of complete train data
    :return test: Pandas dataframe of complete test data
    :return train_text: Pandas series of text comments in train data
    :return test_text: Pandas series of text comments in test data
    :return all_text: train_text and test_text combined together
    '''
    # Load data
    train = pd.read_csv('./data/train.csv').fillna(' ')
    test = pd.read_csv('./data/test.csv').fillna(' ')

    # Take comment data
    train_text = train['comment_text']
    test_text = test['comment_text']
    
    # Combine train and test
    all_text = pd.concat([train_text, test_text])
    
    return train, test, train_text, test_text, all_text
print('hey')

hey


In [7]:
def get_vectorized_features(train_text, test_text, all_text):
    '''
    This function applied tf-idf technique on both words and characters
    in the text comments to feed to Logistic Regression classifier.
    :param train_text: Pandas series containing train comments
    :param test_text: Pandas series containing test comments
    :param all_text: Pnadas series contaning combined train and text comments
    :return combined word and character vectorized features for both
        train and test text data
    '''
    
    # For words
    word_vectorizer = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', 
                                      analyzer='word',token_pattern=r'\w{1,}',
                                      stop_words='english',ngram_range=(1, 1),
                                      max_features=10000)
    word_vectorizer.fit(all_text)
    train_word_features = word_vectorizer.transform(train_text)
    test_word_features = word_vectorizer.transform(test_text)

    # For characters
    char_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='char',
        stop_words='english',
        ngram_range=(2, 6),
        max_features=50000
    )
    char_vectorizer.fit(all_text)
    train_char_features = char_vectorizer.transform(train_text)
    test_char_features = char_vectorizer.transform(test_text)

    # Stack word and character features
    train_features = hstack([train_char_features, train_word_features])
    test_features = hstack([test_char_features, test_word_features])
    
    return train_features, test_features
print('heyyy')

heyyy


In [8]:
def run_classifier(train, test, train_features, test_features):
    '''
    Runs Logistic Regression model for each individual class as one comment
    can have multiple class labels.
    :param train: Pandas dataframe of complete train data
    :param test: Pandas dataframe of complete test data
    :param train_features: Vectorized TFIDF features of text comments in
        the train data
    :param test_features: Vectorized TFIDF features of text comments in
        the test data        
    '''
    
    scores = []
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    
    # Create empty dataframe with each class label to store predictions
    submission = pd.DataFrame.from_dict({'id': test['id']})
    
    # Run for every class
    for class_name in class_names:
        
        # Get labels for current class from train data
        train_target = train[class_name]
        
        # Create a classifier
        classifier = LogisticRegression(C=0.1, solver='sag')

        # Fit the data
        classifier.fit(train_features, train_target)
        # Predict
        submission[class_name] = classifier.predict_proba(test_features)[:, 1]

    print('Total CV score is {}'.format(np.mean(scores)))

    # Save prediction results (probabilities) to a csv file to analyze later
    submission.to_csv('submission_logistic.csv', index=False)
    
print ('heyyyaaa')

heyyyaaa


In [10]:
# Load data
train_data, test_data, train_text, test_text, all_text = load_data()
print('heyyaa')

# Get vectorized features using TFIDF on words as well as characters
train_features, test_features = get_vectorized_features(train_text, test_text, all_text)
print('heyyaa')

# Run the model
run_classifier(train_data, test_data, train_features, test_features)
print('heyyaa')


heyyaa
heyyaa


/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Total CV score is nan
heyyaa
